### Thursday, December 7, 2023

Gonna give this another go ... 

Ok. I was able to download the model, and get this notebook to run. Nice!

### Saturday, December 2, 2023

https://huggingface.co/Intel/neural-chat-7b-v3-1

docker container start hfpt_Oct28

In [1]:
# only target the 4090 ...
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [1]:
!ls /home/rob/Data2/huggingface/transformers

models--Intel--neural-chat-7b-v3-1
models--TheBloke--CodeLlama-34B-Instruct-GPTQ
models--TheBloke--Llama-2-13b-Chat-GPTQ
models--TheBloke--Python-Code-33B-GPTQ
models--facebook--blenderbot-1B-distill
models--google--flan-t5-large
models--gpt2-medium
models--meta-llama--Llama-2-13b-chat-hf
models--meta-llama--Llama-2-13b-hf
models--model_name
tmp0m77pc0x
tmpcjh0h7gn
tmpfcrmwgx2
tmpzafytbf_
version.txt


In [4]:
# # Backup 'Intel/neural-chat-7b-v3-1'
# docker cp c9b676310ea0://home/rob/Data2/huggingface/transformers/models--Intel--neural-chat-7b-v3-1 /home/rob/Data3/huggingface/transformers
# Successfully copied 14.5GB to /home/rob/Data3/huggingface/transformers


In [5]:
import transformers

model_name = 'Intel/neural-chat-7b-v3-1'

In [6]:
!pip show transformers

Name: transformers
Version: 4.35.0.dev0
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /usr/local/lib/python3.8/dist-packages
Requires: requests, numpy, safetensors, tqdm, packaging, regex, filelock, tokenizers, pyyaml, huggingface-hub
Required-by: trl, sentence-transformers, peft, optimum, auto-gptq


In [7]:
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)

In [8]:
%%time

# this loads it to CPU Ram, not GPU VRAM
# model = transformers.AutoModelForCausalLM.from_pretrained(model_name)

# this too loads it to CPU RAM, but then loads it to GPU VRAM 23990 MiB VRAM
# model = transformers.AutoModelForCausalLM.from_pretrained(model_name).to(device)

# This loads it right away to GPU VRAM, but takes up way less space 8252 MiB ... wayy smaller!
# model = transformers.AutoModelForCausalLM.from_pretrained(model_name, load_in_8bit=True)

# This loads it right away to GPU VRAM, but takes up way less space 8252 MiB ... same as above.
model = transformers.AutoModelForCausalLM.from_pretrained(model_name, load_in_8bit=True, torch_dtype=torch.float16)

# This loads it right away to GPU VRAM, but takes up way less space 5392 MiB ... even smaller!
# model = transformers.AutoModelForCausalLM.from_pretrained(model_name, load_in_4bit=True)

# Time to download the model, then load it to CPU RAM
# CPU times: user 1min 49s, sys: 1min 44s, total: 3min 33s
# Wall time: 3h 22min 46s

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

CPU times: user 8.44 s, sys: 2.65 s, total: 11.1 s
Wall time: 6.51 s


In [9]:
def generate_response(system_input, user_input):

    # Format the input using the provided template
    prompt = f"### System:\n{system_input}\n### User:\n{user_input}\n### Assistant:\n"

    # Tokenize and encode the prompt
    inputs = tokenizer.encode(prompt, return_tensors="pt", add_special_tokens=False)

    # Generate a response
    outputs = model.generate(inputs, max_length=1000, num_return_sequences=1)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract only the assistant's response
    return response.split("### Assistant:\n")[-1]

In [10]:
# Example usage
system_input = "You are a math expert assistant. Your mission is to help users understand and solve various math problems. You should provide step-by-step solutions, explain reasonings and give the correct answer."
user_input = "calculate 100 + 520 + 60"
response = generate_response(system_input, user_input)
print(response)

# 28.2s

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/transformers/src/transformers/generation/utils.py:1591: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


 To calculate the sum of 100, 520, and 60, we will follow these steps:

1. Add the first two numbers: 100 + 520
2. Add the result from step 1 to the third number: (100 + 520) + 60

Step 1: Add 100 and 520
100 + 520 = 620

Step 2: Add the result from step 1 to the third number
(620) + 60 = 680

So, the sum of 100, 520, and 60 is 680.


In [ ]:
# expected response
"""
To calculate the sum of 100, 520, and 60, we will follow these steps:

1. Add the first two numbers: 100 + 520
2. Add the result from step 1 to the third number: (100 + 520) + 60

Step 1: Add 100 and 520
100 + 520 = 620

Step 2: Add the result from step 1 to the third number (60)
(620) + 60 = 680

So, the sum of 100, 520, and 60 is 680.
"""